## Available stocks

In [17]:
from dotenv import load_dotenv
import pandas as pd
import requests
import datetime
import os

from sqlalchemy import text

load_dotenv()

TIINGO_API = os.environ["TIINGO_API"]

In [ ]:
headers = {
    'Content-Type': 'application/json'
}
response = requests.get(f"https://api.tiingo.com/tiingo/daily/meta?token={TIINGO_API}&columns=permaTicker,ticker,exchange,isActive", headers=headers).json()

In [ ]:
all_tickers = pd.DataFrame(response)

today = datetime.datetime.now()
all_tickers["created_at"] = today

In [ ]:
all_tickers.to_csv(f"config/supported_tickers_{today}.csv", index=False)

### Export NSYE and NASDAQ tickers

In [1]:
import os
from src.data_ingestion.db.postgres_db import PostgresConfig, PostgresDB


db_config = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
)
db = PostgresDB(config=db_config)


In [ ]:
all_tickers[all_tickers["exchange"].isin(["NASDAQ", "NYSE"]) & all_tickers["isActive"]].to_sql("active_tickers", db.engine, if_exists='append', index=False)

Check database insertion

In [18]:
# Query based
df_all_tickers = pd.read_sql(text("SELECT ticker FROM active_tickers WHERE created_at = :date_filter"), db.engine, params={"date_filter": pd.read_sql(text("SELECT max(created_at) FROM active_tickers"), db.engine)["max"].to_list()[0]})

In [ ]:
# In our object. To be decoupled
from src.data_ingestion.downloader.config.tiingo_ticker_downloader import TiingoDownloader

dwn = TiingoDownloader(engine=db.engine)  
dwn.available_tickers(date=None)

## End day OHLC

In [9]:
import datetime

ticker = "aapl"
yesteday_close_date = datetime.datetime(2024, 7 ,18)

In [10]:
from src.data_ingestion.downloader.config.tiingo_ticker_downloader import TiingoDownloader

dwn = TiingoDownloader(engine=db.engine)  
tickers = dwn.available_tickers(date=None)["ticker"].to_list()

In [12]:
df = dwn.get_prices(tickers[0:5], yesteday_close_date)

[{'date': '2024-07-18T00:00:00.000Z', 'close': 0.15, 'high': 0.15, 'low': 0.15, 'open': 0.15, 'volume': 0, 'adjClose': 0.15, 'adjHigh': 0.15, 'adjLow': 0.15, 'adjOpen': 0.15, 'adjVolume': 0, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 25.28, 'high': 25.32, 'low': 25.239, 'open': 25.2975, 'volume': 14716, 'adjClose': 25.28, 'adjHigh': 25.32, 'adjLow': 25.239, 'adjOpen': 25.2975, 'adjVolume': 14716, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 47.51, 'high': 48.56, 'low': 47.38, 'open': 48.34, 'volume': 7263517, 'adjClose': 47.51, 'adjHigh': 48.56, 'adjLow': 47.38, 'adjOpen': 48.34, 'adjVolume': 7263517, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 13.61, 'high': 13.87, 'low': 13.51, 'open': 13.59, 'volume': 387216, 'adjClose': 13.61, 'adjHigh': 13.87, 'adjLow': 13.51, 'adjOpen': 13.59, 'adjVolume': 387216, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:

In [13]:
df

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2024-07-18T00:00:00.000Z,0.15,0.15,0.150,0.1500,0,0.15,0.15,0.150,0.1500,0,0.0,1.0
0,2024-07-18T00:00:00.000Z,25.28,25.32,25.239,25.2975,14716,25.28,25.32,25.239,25.2975,14716,0.0,1.0
0,2024-07-18T00:00:00.000Z,47.51,48.56,47.380,48.3400,7263517,47.51,48.56,47.380,48.3400,7263517,0.0,1.0
0,2024-07-18T00:00:00.000Z,13.61,13.87,13.510,13.5900,387216,13.61,13.87,13.510,13.5900,387216,0.0,1.0
0,2024-07-18T00:00:00.000Z,26.47,27.76,26.360,27.7000,1932964,26.47,27.76,26.360,27.7000,1932964,0.0,1.0


In [15]:
df = dwn.get_daily_prices(tickers[0:5], yesteday_close_date)

[{'date': '2024-07-18T00:00:00.000Z', 'close': 0.15, 'high': 0.15, 'low': 0.15, 'open': 0.15, 'volume': 0, 'adjClose': 0.15, 'adjHigh': 0.15, 'adjLow': 0.15, 'adjOpen': 0.15, 'adjVolume': 0, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 25.28, 'high': 25.32, 'low': 25.239, 'open': 25.2975, 'volume': 14716, 'adjClose': 25.28, 'adjHigh': 25.32, 'adjLow': 25.239, 'adjOpen': 25.2975, 'adjVolume': 14716, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 47.51, 'high': 48.56, 'low': 47.38, 'open': 48.34, 'volume': 7263517, 'adjClose': 47.51, 'adjHigh': 48.56, 'adjLow': 47.38, 'adjOpen': 48.34, 'adjVolume': 7263517, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:00.000Z', 'close': 13.61, 'high': 13.87, 'low': 13.51, 'open': 13.59, 'volume': 387216, 'adjClose': 13.61, 'adjHigh': 13.87, 'adjLow': 13.51, 'adjOpen': 13.59, 'adjVolume': 387216, 'divCash': 0.0, 'splitFactor': 1.0}]
[{'date': '2024-07-18T00:00:

In [16]:
df

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2024-07-18T00:00:00.000Z,0.15,0.15,0.150,0.1500,0,0.15,0.15,0.150,0.1500,0,0.0,1.0
0,2024-07-18T00:00:00.000Z,25.28,25.32,25.239,25.2975,14716,25.28,25.32,25.239,25.2975,14716,0.0,1.0
0,2024-07-18T00:00:00.000Z,47.51,48.56,47.380,48.3400,7263517,47.51,48.56,47.380,48.3400,7263517,0.0,1.0
0,2024-07-18T00:00:00.000Z,13.61,13.87,13.510,13.5900,387216,13.61,13.87,13.510,13.5900,387216,0.0,1.0
0,2024-07-18T00:00:00.000Z,26.47,27.76,26.360,27.7000,1932964,26.47,27.76,26.360,27.7000,1932964,0.0,1.0
